```
███████  █████  ███████ ████████ ████████ ██████   █████  ███    ██ ███████ ███████ ███████ ██████  
██      ██   ██ ██         ██       ██    ██   ██ ██   ██ ████   ██ ██      ██      ██      ██   ██ 
█████   ███████ ███████    ██       ██    ██████  ███████ ██ ██  ██ ███████ █████   █████   ██████  
██      ██   ██      ██    ██       ██    ██   ██ ██   ██ ██  ██ ██      ██ ██      ██      ██   ██ 
██      ██   ██ ███████    ██       ██    ██   ██ ██   ██ ██   ████ ███████ ██      ███████ ██   ██ 

# Demo 1 : Copy From MSSQL 2022 to MSSQL 2025

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\romai\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

## Working on TPCH SF10 ORDERS (15M rows)

### MSSQL 2022 Instance to MSSQL 2025 Instance

In [2]:
#!connect mssql --kernel-name SS2025_tpch_test --connection-string "Data Source=localhost\SS2025;Initial Catalog=tpch_test;Integrated Security=True;Connect Timeout=120;Encrypt=True;Trust Server Certificate=True;Application Name=NoteBook;Application Intent=ReadWrite;Command Timeout=10800"

Kernel added: #!sql-SS2025_tpch_test

#### Target orders with a clustered ColumnStoreIndex

In [3]:
TRUNCATE TABLE orders;
SELECT count(*) count_target_table_rows from orders;

(1 ligne affectée)

count_target_table_rows
0


The next cell is a powershell call of FastTransfer.exe. FastTransfer also exists in a Linux AMD64 and Linux ARM64 edition with the same arguments.

You can notice the **negative degree of parallelism** : -2 ==> FastTransfer will take 1/2 of total number of cores of the host as the computed degree of parallelism.
The method use to dispatch data in several channels/chunks is **Ntile**. A method that will spread data using a distribute key column and an homogeneous number of rows between channels/chunks

In [4]:

cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch10_collation_bin2" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_15M" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ntile" `
--distributekeycolumn "o_orderkey" `
--degree -2 `
--runid "MSS22_to_MSS25"

2025-09-25T17:37:48.968 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (24 day(s) left).
2025-09-25T17:37:48.982 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders -|- Starting
2025-09-25T17:37:48.984 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-25T17:37:48.984 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders -|- OS : Microsoft Windows 10.0.26100
2025-09-25T17:37:48.984 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders -|- Process ID : 3348
2025-09-25T17:37:48.984 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders -|- Provided Run ID : MSS22_to_MSS25
2025-09-25T17:37:48.984 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- I

In [5]:
SELECT FORMAT(count(*),'###,###,###') count_target_table_rows from orders;

(1 ligne affectée)

count_target_table_rows
"15,000,000"


#### Target orders_4_no_index : Heap Table

In [6]:
DROP TABLE IF EXISTS orders_4_no_index;

SELECT * 
INTO orders_4_no_index
FROM orders
WHERE 1=0;

SELECT FORMAT(count(*),'###,###,###') count_orders_4_no_index from orders_4_no_index;

(0 lignes affectées)

(1 ligne affectée)

count_orders_4_no_index
""


In [7]:

cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch10_collation_bin2" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_15M" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ntile" `
--distributekeycolumn "o_orderkey" `
--degree -2 `
--runid "MSS22_to_MSS25"

2025-09-25T17:40:33.331 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (24 day(s) left).
2025-09-25T17:40:33.341 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-09-25T17:40:33.341 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-25T17:40:33.341 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-09-25T17:40:33.341 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 36288
2025-09-25T17:40:33.341 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Provided Run ID : MSS22_to_MSS25
2025-

In [8]:

SELECT FORMAT(count(*),'###,###,###') count_orders_4_no_index from orders_4_no_index;

(1 ligne affectée)

count_orders_4_no_index
"15,000,000"


In [9]:
SELECT TOP 5 * from orders_4_no_index;

(5 lignes affectées)

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment,ct
108688866,68986,O,45015.28,1999-04-30 00:00:00Z,1-URGENT,Clerk#000002476,0,deposits along the furiously regular deposits sublate quickly across th,<null>
108784452,1475186,O,227236.70,1999-04-30 00:00:00Z,1-URGENT,Clerk#000004829,0,carefully final foxes wake. fluffily final sheaves nag carefully quickly,<null>
108853858,128426,O,252434.36,1999-04-30 00:00:00Z,1-URGENT,Clerk#000005029,0,express deposits haggle idly above the theodolites. slyly regu,<null>
108898407,392066,O,370344.13,1999-04-30 00:00:00Z,1-URGENT,Clerk#000003188,0,carefully even gifts among the deposits cajole slyly acros,<null>
108901569,1132472,O,334924.30,1999-04-30 00:00:00Z,1-URGENT,Clerk#000005560,0,furiously even somas slee,<null>


In [10]:
DROP TABLE IF EXISTS orders_6_classic_index;

SELECT * 
INTO orders_6_classic_index
FROM orders
WHERE 1=0;

CREATE CLUSTERED INDEX PK_orders_6_classic_index ON  orders_6_classic_index (o_orderkey);

SELECT count(*) count_orders_6_classic_index from orders_6_classic_index;

(0 lignes affectées)

(1 ligne affectée)

count_orders_6_classic_index
0


In [3]:
TRUNCATE TABLE orders_6_classic_index;
GO

INSERT INTO orders_6_classic_index WITH (TABLOCK)
SELECT * from orders;
GO

Commandes exécutées.

(15000000 lignes affectées)

First, just to compare performance with a local copy of data inside SQL Server for this case : Target with a classic clustered index, let's do the copy using TSQL only

In [4]:
TRUNCATE TABLE orders_6_classic_index;
GO

INSERT INTO orders_6_classic_index ([o_orderkey], [o_custkey], [o_orderstatus], [o_totalprice], [o_orderdate], [o_orderpriority], [o_clerk], [o_shippriority], [o_comment])
SELECT [o_orderkey], [o_custkey], [o_orderstatus], [o_totalprice], [o_orderdate], [o_orderpriority], [o_clerk], [o_shippriority], [o_comment]
from [LOCALHOST].[tpch10_collation_bin2].dbo.orders_15M;

Commandes exécutées.

(15000000 lignes affectées)

Now let's do the copy using FastTransfer still using the same target table with a classic clustered index.

In [11]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch10_collation_bin2" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_15M" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_6_classic_index" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ntile" `
--distributekeycolumn "o_orderkey" `
--degree -2 `
--runid "MSS22_to_MSS25"

2025-09-25T17:46:04.072 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_6_classic_index -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (24 day(s) left).
2025-09-25T17:46:04.082 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_6_classic_index -|- Starting
2025-09-25T17:46:04.082 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_6_classic_index -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-25T17:46:04.082 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_6_classic_index -|- OS : Microsoft Windows 10.0.26100
2025-09-25T17:46:04.082 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_6_classic_index -|- Process ID : 43720
2025-09-25T17:46:04.082 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_6_classic_index -|- Provided

#### Target orders with a primary key

In [16]:
DROP TABLE IF EXISTS orders_7_primarykey;

SELECT * 
INTO orders_7_primarykey
FROM orders
WHERE 1=0;

ALTER TABLE  orders_7_primarykey ADD CONSTRAINT PK_orders_7_primarykey PRIMARY KEY CLUSTERED (o_orderkey);

SELECT count(*) count_orders_7_primarykey from orders_7_primarykey;

(0 lignes affectées)

(1 ligne affectée)

count_orders_7_primarykey
0


In [17]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch10_collation_bin2" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_15M" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_7_primarykey" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ntile" `
--distributekeycolumn "o_orderkey" `
--degree -2 `
--runid "MSS22_to_MSS25"

2025-09-01T22:34:09.768 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- FastTransfer - running in trial mode – will expires on 2025‑09‑19 (17 day(s) left).
2025-09-01T22:34:09.778 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Starting
2025-09-01T22:34:09.778 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- FastTransfer Version : 0.13.9.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-01T22:34:09.778 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- OS : Microsoft Windows 10.0.26100
2025-09-01T22:34:09.778 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Process ID : 30716
2025-09-01T22:34:09.778 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Provided Run ID : MSS22_to_

In [18]:
SELECT FORMAT(count(*),'###,###,###') count_orders_7_primarykey from orders_7_primarykey;

(1 ligne affectée)

count_orders_7_primarykey
"15,000,000"


#### Target orders with a primary key and one secondary index

In [ ]:
CREATE INDEX IX_o_custkey ON orders_7_primarykey(o_custkey);

Commandes exécutées.

In [ ]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch10_collation_bin2" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_15M" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_7_primarykey" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ntile" `
--distributekeycolumn "o_orderkey" `
--degree -2 `
--runid "MSS22_to_MSS25"

2025-06-13T22:56:28.897 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Starting
2025-06-13T22:56:28.905 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- FastTransfer Version : 0.13.4.0 Architecture : X64 - Framework : .NET 8.0.15
2025-06-13T22:56:28.906 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- OS : Microsoft Windows 10.0.26100
2025-06-13T22:56:28.906 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Process ID : 28124
2025-06-13T22:56:28.906 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Provided Run ID : MSS22_to_MSS25
2025-06-13T22:56:28.906 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Source Connection Type : mssql
2025-06-13T22:56:28.906 +02:00 -|- FastTransfer

#### Target orders with a primary key and 2 secondary index

Check sp_whoisactive results during load

In [ ]:
CREATE INDEX IX_o_orderdate ON orders_7_primarykey(o_orderdate);

Commandes exécutées.

In [ ]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch10_collation_bin2" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_15M" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_7_primarykey" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ntile" `
--distributekeycolumn "o_orderkey" `
--degree -2 `
--runid "MSS22_to_MSS25"

2025-06-13T22:56:50.743 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Starting
2025-06-13T22:56:50.752 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- FastTransfer Version : 0.13.4.0 Architecture : X64 - Framework : .NET 8.0.15
2025-06-13T22:56:50.752 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- OS : Microsoft Windows 10.0.26100
2025-06-13T22:56:50.752 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Process ID : 50964
2025-06-13T22:56:50.752 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Provided Run ID : MSS22_to_MSS25
2025-06-13T22:56:50.752 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_7_primarykey -|- Source Connection Type : mssql
2025-06-13T22:56:50.752 +02:00 -|- FastTransfer

| MSSQL 2022 ==> MSSQL 2025  ORDERS (15M Rows x 9 columns)				| Elapsed Time (seconds)|Cells/sec 	|
|-----------------------------------------------------------------------|:---------------------:|----------:|
| FastTransfer Load (orders_3 clustered columnstore)      				| 8.2      				|16_799_794 |
| FastTransfer Load (orders_4 heap) 									| 5.5      				|25_106_464 |
| FastTransfer Load (orders_6 cluster index ) 							| 7.7      				|17_921_206 |
| FastTransfer Load (orders_7 cluster primarykey) 						| 7.4      				|18_528_328 |
| FastTransfer Load (orders_7 cluster primarykey + 1 secondary index) 	|19.2      				| 7_091_346 |
| FastTransfer Load (orders_7 cluster primarykey + 2 secondary indexes) |58.1      				| 2_328_917 |

## Other parallel technics

### method="DataDriven" using a standard column (or an expression) for the distributekeycolumn parameter

#### DataDriven using year(o_orderdate) as the distributekeycolumn

In [12]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch10_collation_bin2" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_15M" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "DataDriven" `
--distributekeycolumn "year(o_orderdate)" `
--degree 10 `
--runid "MSS22_to_MSS25"

2025-09-25T17:54:35.944 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (24 day(s) left).
2025-09-25T17:54:35.955 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-09-25T17:54:35.956 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-25T17:54:35.956 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-09-25T17:54:35.956 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 40432
2025-09-25T17:54:35.956 +02:00 -|- FastTransfer -|- MSS22_to_MSS25 -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Provided Run ID : MSS22_to_MSS25
2025-

#### DataDriven using partitions as the distributekeycolumn

In [13]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch_test" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_part" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "DataDriven" `
--distributekeycolumn "`$PARTITION.PF_DATE(o_orderdate)" `
--degree 7 `
--runid "MSS22_to_MSS25_Datadriven_Partition"

2025-09-25T17:56:19.130 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (24 day(s) left).
2025-09-25T17:56:19.139 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-09-25T17:56:19.139 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-25T17:56:19.139 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-09-25T17:56:19.139 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 17796
2025-09-25T17:56:19.139 +02:00 -|- FastTransfer -|- MSS2

#### DataDriven using partitions as the distributekeycolumn and a custom datadrivenquery (simple)

In [ ]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch_test" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_part" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "DataDriven" `
--distributekeycolumn "`$PARTITION.PF_DATE(o_orderdate)" `
--datadrivenquery "select value from generate_series(6,8)" `
--degree 7 `
--runid "MSS22_to_MSS25_Datadriven_Partition"

2025-09-25T17:58:27.113 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (24 day(s) left).
2025-09-25T17:58:27.122 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-09-25T17:58:27.122 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-25T17:58:27.122 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-09-25T17:58:27.122 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 59280
2025-09-25T17:58:27.122 +02:00 -|- FastTransfer -|- MSS2

#### DataDriven using partitions as the distributekeycolumn and a custom datadrivenquery (a little more complex)

In [5]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch_test" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_part" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "DataDriven" `
--distributekeycolumn "`$PARTITION.PF_DATE(o_orderdate)" `
--datadrivenquery "select boundary_id partid 
from 
  sys.partition_range_values prv 
  inner join sys.partition_functions pf on prv.function_id=pf.function_id
where 
  pf.name='PF_DATE'  
  and cast(prv.value as date)  > '1996-01-01' and cast(prv.value as date) <= '1999-01-01'" `
--degree 7 `
--runid "MSS22_to_MSS25_Datadriven_Partition"

2025-06-15T17:34:11.839 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑07‑14 (28 day(s) left).
2025-06-15T17:34:11.850 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-06-15T17:34:11.851 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.6.0 Architecture : X64 - Framework : .NET 8.0.16
2025-06-15T17:34:11.851 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-06-15T17:34:11.851 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Datadriven_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 41348
2025-06-15T17:34:11.851 +02:00 -|- FastTransfer -|- MSS22

### Using RangeId as parallel dispatch method and a filtered query as source

In [7]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch_test" `
--sourcetrusted `
--query "SELECT * FROM orders_part WHERE o_orderdate between '1996-01-01' AND '1999-12-31'" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "RangeId" `
--distributekeycolumn "o_orderkey" `
--degree 7 `
--runid "MSS22_to_MSS25_RangeId_and_filtered_data"

2025-06-15T17:34:51.460 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_RangeId_and_filtered_data -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑07‑14 (28 day(s) left).
2025-06-15T17:34:51.470 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_RangeId_and_filtered_data -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-06-15T17:34:51.470 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_RangeId_and_filtered_data -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.6.0 Architecture : X64 - Framework : .NET 8.0.16
2025-06-15T17:34:51.470 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_RangeId_and_filtered_data -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-06-15T17:34:51.470 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_RangeId_and_filtered_data -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 37956
2025-06-15T17:34:51.470 +02:00 -

In [9]:
SELECT count(*) from orders_4_no_index

(1 ligne affectée)

(Aucun nom de colonne)
15000000


## Using physloc

In [10]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch10_collation_bin2" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_15M" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Physloc" `
--degree 12 `
--runid "MSS22_to_MSS25_Physloc"

2025-09-25T00:20:46.328 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (25 day(s) left).
2025-09-25T00:20:46.337 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-09-25T00:20:46.338 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-25T00:20:46.338 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-09-25T00:20:46.338 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 45768
2025-09-25T00:20:46.338 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc -|- INFORMATION -|- tpch_test.dbo.orders_4_no_

Physloc on a partitioned table :

In [11]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "mssql" `
--sourceserver "localhost" `
--sourcedatabase "tpch_test" `
--sourcetrusted `
--sourceschema "dbo" `
--sourcetable "orders_part" `
`
--targetconnectiontype "msbulk" `
--targetserver "localhost\SS2025" `
--targettrusted `
--targetdatabase "tpch_test" `
--targetschema "dbo" `
--targettable "orders_4_no_index" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Physloc" `
--degree 7 `
--runid "MSS22_to_MSS25_Physloc_Partition"

2025-09-25T00:23:30.744 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (25 day(s) left).
2025-09-25T00:23:30.753 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Starting
2025-09-25T00:23:30.753 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-25T00:23:30.753 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- OS : Microsoft Windows 10.0.26100
2025-09-25T00:23:30.753 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Physloc_Partition -|- INFORMATION -|- tpch_test.dbo.orders_4_no_index -|- Process ID : 39652
2025-09-25T00:23:30.753 +02:00 -|- FastTransfer -|- MSS22_to_MSS25_Phys